In [1]:
#define a function to calculate the Euclidean Distance

import math 
def euclideandistance(instance1,instance2,length):
    distance=0
    for x in range(1,length):
        #print(instance1[x],instance2[x])
        distance+=pow((instance1[x]-instance2[x]),2)
        #print(distance)
    return math.sqrt(distance)

In [2]:
#write a function to find the neighbourhood
import operator

def getneighborhood(trainingdata,testinstance,k):
    distance=[]
    length=len(testinstance)
    #print("Length of testinstance:",length)
    #print("Length trainingSet",len(trainingdata))
    for x in range(len(trainingdata)):
        dist=euclideandistance(trainingdata[x],testinstance,length)
        distance.append((trainingdata[x],dist))
    
    distance.sort(key=operator.itemgetter(1))
    neighbor=[]
    #print(distance)
    for x in range(k):
        neighbor.append(distance[x][0])
    #print("Neighbors of the test instance",neighbor)
    return neighbor
    


In [3]:
#write a function to get the output class from the neighbors
import operator

def getoutputclass(neighbors):
    outputclass={}
    for x in range(len(neighbors)):
        response=neighbors[x][0]
        if response in outputclass:
            outputclass[response]+=1
        else:
            outputclass[response]=1
    sortedclass=sorted(outputclass.items(), key=operator.itemgetter(1),reverse=True)
    #print("Class for the test instance",outputclass.items())
    return sortedclass[0][0]

In [4]:
def getaccuracy(testinstance, prediction):
    correct=0
#     print("Test instance in Accuracy",testinstance)
#     print("Test instance 19",testinstance[19])
#     print("Prediction",prediction)
    for x in range(len(testinstance)):
        if testinstance[x][0] == prediction[x]:
            correct+=1
            #print(correct)
    return (correct/len(testinstance))*100 

In [5]:
#function to split the dataset into train and test

def splitdataset(dataset,trainingdata=[], testdata=[]):
    for x in range(len(dataset)):
        if x<int(len(dataset)*0.8):
            trainingdata.append(dataset[x])
        else:
            testdata.append(dataset[x])
    #print(x)

In [12]:
import csv
import numpy as np
import pandas as pd

def main():
    #define variables and other parameters
    
    trainingSet=[]
    testSet=[]
    dataSet=[]
    data=open("D:\\NEU Courses\\Predictive Anaytics\\Final Project\\NYPD_Complaint_Data_Historic.csv",'r')

    df_data=pd.read_csv(data)

    #select only the relevant columns
    
    cols_mode = ['OFNS_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'BORO_NM', 
                 'LOC_OF_OCCUR_DESC', 'JURIS_DESC', 'PREM_TYP_DESC', 'SUSP_RACE', 'SUSP_SEX', 'VIC_RACE', 'VIC_SEX']

    updated_df_train=df_data[cols_mode]
    
    #check for missing values and replace by most frequent
    for col in cols_mode: 
        updated_df_train[col].fillna(updated_df_train[col].value_counts().idxmax(), inplace=True)
    
    
    #randomly select rows from the dataset
    subset_indices = np.random.randint(low=0, high=len(updated_df_train), size=10000)
    selected_df_train = updated_df_train.iloc[subset_indices]
    
    #create dummy variables
    dummy_df=pd.get_dummies(selected_df_train, columns=['OFNS_DESC', 'CRM_ATPT_CPTD_CD', 'BORO_NM', 'LOC_OF_OCCUR_DESC',
                                                       'JURIS_DESC', 'PREM_TYP_DESC', 'SUSP_RACE', 'SUSP_SEX', 
                                                       'VIC_RACE', 'VIC_SEX'])


    #add rows in a list and split train and test dataset
    
    for x in subset_indices:
        a=list(dummy_df.loc[x])
        dataSet.append(a)
        
        
    #convert the data elements (except Crime label - target variable) into float type 
    for i in range(len(dataSet)):
        a=dataSet[i]
        for j in range(1,len(a)):
            #print(i,j)
            if (type(a[j])==str):
                a[j]=0.0
            else:
                b=a[j].astype(float)
                dataSet[i][j]=b
            
    #print(len(dataSet))
    splitdataset(dataSet,trainingSet,testSet)
    print("Training Data dimensions:",len(trainingSet),",",len(trainingSet[0]))
    print("Test Data:",len(testSet),",",len(testSet[0]))
#     print("Sample Training Set:",trainingSet[0:5])
#     print("Sample Test Set:",testSet[0:5])
    
    #call functions for each test row and classify the output
    #generate output
    prediction=[]
    k=5
    for x in range(len(testSet)):
        neighbors=getneighborhood(trainingSet,testSet[x],k)
        #print(neighbors)
        response=getoutputclass(neighbors)
        #print(response)
        prediction.append(response)
        #print("Predicted:",response,"Actual:",testSet[x][-1])
    print("Predicted Crimes for the test dataset:",prediction[:10])
    accuracy=getaccuracy(testSet,prediction)
    print("Accuracy of the KNN model:",accuracy)

main()

Training Data dimensions: 8000 , 162
Test Data: 2000 , 162
Predicted Crimes for the test dataset: ['MISDEMEANOR', 'MISDEMEANOR', 'MISDEMEANOR', 'FELONY', 'MISDEMEANOR', 'FELONY', 'MISDEMEANOR', 'FELONY', 'MISDEMEANOR', 'FELONY']
Accuracy of the KNN model: 87.85
